## **Session**

- https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#executing-with-an-orm-session
- https://www.sqlalchemy.org/

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
import os
import sqlalchemy
import os
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from sqlalchemy.orm import Session


In [3]:
_ = load_dotenv(find_dotenv())

In [4]:
_

True

In [5]:
# Get connection string from environment variable
conn_str = os.environ.get('CONNECTION_STRING')

# Optional: Handle missing connection string
if not conn_str:
    raise ValueError("CONNECTION_STRING environment variable not set.")

# Create SQLAlchemy engine
engine: Engine = create_engine(conn_str,
    echo=False,
    pool_size=10,
    max_overflow=20,
    pool_pre_ping=True,  # Verify connection is alive before use
    pool_recycle=3600,   # Reconnect after 1 hour (helps with Neon timeouts)
)

In [6]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

# --- Step 2: Define Base & Model ---
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(50))
    email = Column(String(50), unique=True)

# --- Step 3: Create the Table ---
Base.metadata.create_all(bind=engine)

# --- Step 4: Set up Session ---
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db = SessionLocal()

# --- Step 5: Add a User ---
new_user = User(name="Alice", email="alice@example.com")
db.add(new_user)
db.commit()
db.refresh(new_user)

print("User created with ID:", new_user.id)
db.close()

User created with ID: 1
